<a href="https://colab.research.google.com/github/parag-ghungrudkar/Coursera_Capstone/blob/main/Coursera_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM Data Science Capstone Project
###### The Battle of neighborhood: The business object of this project is to cluster neighborhood with similar venues together. Further explanation will be detailed in this ...placeholder

###week 2:
Creating a notebook for the project. I am using google colab for the purpose.

In [ ]:
! pip install geocoder

In [ ]:
#importing the required libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from bs4 import BeautifulSoup
import folium # map rendering library

In [ ]:
#printing the required string
print('Hello capstone project coursera')

Hello capstone project coursera


###Week 3
## Part 1:

1. Created pandas dataframe for the neighborhood in Toronto using web scrapping
2. Found nearby venues for each neighborhood using FourSquare API
3. One hot encoding for categories
4. Clustering analysis of the neighborhood

In [ ]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronto_neighbourhood = pd.read_html(url, match = "Neighbourhood", flavor = 'bs4')[0]
Toronto_neighbourhood.head()

In [ ]:
Toronto_neighbourhood['Borough'].replace('Not assigned', np.NaN, inplace = True)
Toronto_neighbourhood.head()

In [ ]:
Toronto_neighbourhood.dropna(axis= 0, inplace=True)
Toronto_neighbourhood.reset_index(inplace= True, drop = True)
Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
Toronto_neighbourhood['Neighbourhood'].str.count('Not assigned').sum()

0

####week 3
##Part 2:

In [ ]:
'''
for row in Toronto_neighbourhood['Postal Code']:
  lat_lng_coords = None
  while(lat_lng_coords is None): 
    g = geocoder.google('{}, Toronto, Ontario'.format(row))
    lat_lng_coords = g.latlng
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  lat_lon = lat_lon.append({'Latitude':latitude,'Longitude':longitude},ignore_index = True )'''


In [ ]:
url = 'https://raw.githubusercontent.com/parag-ghungrudkar/Coursera_Capstone/main/Geospatial_Coordinates.csv'
lat_lon = pd.read_csv(url)
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
final_df = pd.merge(Toronto_neighbourhood,lat_lon,on = 'Postal Code')
final_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [ ]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
CLIENT_ID = '0GNGY4JRBUZ32X5431M5XKQKOP2RPGGC2HGQQDGHZNGPC2TF' # your Foursquare ID
CLIENT_SECRET = '5QUO4ZOXLCHHRZXSHT4E4JRRMYEE0TOYVWMVT0Y4RDOXXX1R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0GNGY4JRBUZ32X5431M5XKQKOP2RPGGC2HGQQDGHZNGPC2TF
CLIENT_SECRET:5QUO4ZOXLCHHRZXSHT4E4JRRMYEE0TOYVWMVT0Y4RDOXXX1R


In [ ]:
final_df.loc[0, 'Neighbourhood']

'Parkwoods'

In [ ]:
neighbourhood_latitude = final_df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = final_df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = final_df.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [ ]:
LIMIT = 100 
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [ ]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603fe8ad17b3c815c9dd4482'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'location': {'address': 'Toronto',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 245,
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'lat

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# type your answer here
final_venues = getNearbyVenues(names=final_df['Neighbourhood'],
                                   latitudes=final_df['Latitude'],
                                   longitudes=final_df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [ ]:
print(final_venues.shape)
final_venues.head()

(2121, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [ ]:
final_venues.to_csv('file1.csv')

In [107]:
final_df1 = final_df.copy()

In [109]:
final_df1.explode('Neighbourhood').reset_index(drop=True)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [112]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [114]:
exploded_final_df = explode(final_df1.assign(Neighbourhood=final_df1.Neighbourhood.str.split(',')), 'Neighbourhood')

In [116]:
print('the shape of orignal final_df is {}, the shape of exploded final df is {}'.format(final_df.shape, exploded_final_df.shape))

the shape of orignal final_df is (103, 5), the shape of exploded final df is (217, 5)


In [117]:
final_venues1 = getNearbyVenues(names=exploded_final_df['Neighbourhood'],
                                   latitudes=exploded_final_df['Latitude'],
                                   longitudes=exploded_final_df['Longitude'])

Parkwoods
Victoria Village
Regent Park
 Harbourfront
Lawrence Manor
 Lawrence Heights
Queen's Park
 Ontario Provincial Government
Islington Avenue
 Humber Valley Village
Malvern
 Rouge
Don Mills
Parkview Hill
 Woodbine Gardens
Garden District
 Ryerson
Glencairn
West Deane Park
 Princess Gardens
 Martin Grove
 Islington
 Cloverdale
Rouge Hill
 Port Union
 Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
 Bloordale Gardens
 Old Burnhamthorpe
 Markland Wood
Guildwood
 Morningside
 West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
 Wilson Heights
 Downsview North
Thorncliffe Park
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Scarborough Village
Fairview
 Henry Farm
 Oriole
Northwood Park
 York University
East Toronto
 Broadview North (Old East York)
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
Kennedy Park
 Ionview
 East Birchmo

In [118]:
final_venues1.shape

(4325, 7)

In [119]:
toronto_df = exploded_final_df[exploded_final_df['Borough'].str.contains("Toronto")]
toronto_df.head()

,Borough,Latitude,Longitude,Postal Code,Neighbourhood
2,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park
3,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront
6,Downtown Toronto,43.662301,-79.389494,M7A,Queen's Park
7,Downtown Toronto,43.662301,-79.389494,M7A,Ontario Provincial Government
15,Downtown Toronto,43.657162,-79.378937,M5B,Garden District


In [121]:
toronto_df['Borough'].value_counts()

Downtown Toronto    39
Central Toronto     18
West Toronto        13
East Toronto         8
Name: Borough, dtype: int64

In [122]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'])

Regent Park
 Harbourfront
Queen's Park
 Ontario Provincial Government
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
North Toronto West
 Lawrence Park
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
R

In [123]:
toronto_venues.shape

(3147, 7)

In [125]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [161]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,95,95,95,95,95,95
Bathurst Quay,16,16,16,16,16,16
Cabbagetown,46,46,46,46,46,46
Chinatown,64,64,64,64,64,64
Deer Park,15,15,15,15,15,15
...,...,...,...,...,...,...
Summerhill West,15,15,15,15,15,15
The Annex,21,21,21,21,21,21
The Danforth West,43,43,43,43,43,43


In [138]:
neighbourhood_venues = toronto_venues.groupby('Neighbourhood').count().mean()

In [139]:
neighbourhood_venues.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,95,95,95,95,95,95
Bathurst Quay,16,16,16,16,16,16
Cabbagetown,46,46,46,46,46,46
Chinatown,64,64,64,64,64,64
Deer Park,15,15,15,15,15,15


In [141]:
neighbourhood_venues['Venue']['Venue'].mean()

41.96

In [155]:
df_filtered =  neighbourhood_venues[neighbourhood_venues['Venue'] <= 10]

In [156]:
df_filtered.reset_index(inplace=True)

In [157]:
df_filtered

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Forest Hill Road Park,4,4,4,4,4,4
1,Davisville North,8,8,8,8,8,8
2,Forest Hill North & West,4,4,4,4,4,4
3,Lawrence Park,4,4,4,4,4,4
4,Rosedale,4,4,4,4,4,4
5,Roselawn,2,2,2,2,2,2
6,The Beaches,4,4,4,4,4,4


In [159]:
empty_neighbourhood = df_filtered['Neighbourhood'].tolist()
empty_neighbourhood

[' Forest Hill Road Park',
 'Davisville North',
 'Forest Hill North & West',
 'Lawrence Park',
 'Rosedale',
 'Roselawn',
 'The Beaches']

In [160]:
toronto_venues = toronto_venues[~toronto_venues['Neighbourhood'].isin(empty_neighbourhood)]

In [153]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] =toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot =toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [154]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Adelaide,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.021053,0.0,0.0,0.010526,0.010526,0.000000,0.021053,0.0,0.000000,0.0,0.000000,0.031579,0.000000,0.010526,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.021053,0.0000,0.010526,0.010526,0.000000,0.000000,0.010526,0.010526,0.021053,...,0.021053,0.0,0.00,0.0,0.0,0.010526,0.000000,0.000000,0.010526,0.000000,0.010526,0.00,0.0,0.0,0.0,0.021053,0.0,0.000000,0.0,0.021053,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.031579,0.010526,0.0,0.000000,0.0,0.000000,0.00,0.021053,0.000000,0.000000,0.000000,0.0,0.000000
1,Bathurst Quay,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0625,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Cabbagetown,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.065217,0.021739,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.021739,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.021739,0.0,0.000000,0.0,0.021739,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Chinatown,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.015625,0.000000,0.0,0.000000,0.0,0.000000,0.031250,0.000000,0.046875,0.0,0.00,0.0,0.015625,0.015625,0.000000,0.015625,0.0,0.0,0.0000,0.000000,0.0000,0.000000,0.015625,0.000000,0.000000,0.000000,0.015625,0.015625,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.000000,0.0,0.015625,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.046875,0.000000,0.046875,0.015625,0.0,0.000000
4,Deer Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.066667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.066667,0.0,0.066667,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,